# Delta Lake Lab 
## Unit 9: Table Restore + Storage optimization with OPTIMIZE and VACUUM

In the previous unit we-
1. Learned about table clones

In this unit, we will learn about-
1. Table storage optimization with the OPTIMIZE command
2. Restoring a table to a prior version
2. Table storage optimization with the VACUUM command


### 1. Imports

In [1]:
import pandas as pd

from pyspark.sql.functions import month, date_format
from pyspark.sql.types import IntegerType
from pyspark.sql import SparkSession

from delta.tables import *

import warnings
warnings.filterwarnings('ignore')

### 2. Create a Spark session powered by Cloud Dataproc 

In [2]:
spark = SparkSession.builder.appName('Loan Analysis').getOrCreate()
spark

23/12/03 00:05:08 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


### 3. Declare variables

In [3]:
project_id_output = !gcloud config list --format "value(core.project)" 2>/dev/null
PROJECT_ID = project_id_output[0]
print("PROJECT_ID: ", PROJECT_ID)

PROJECT_ID:  delta-lake-diy-lab


In [4]:
project_name_output = !gcloud projects describe $PROJECT_ID | grep name | cut -d':' -f2 | xargs
PROJECT_NAME = project_name_output[0]
print("PROJECT_NAME: ", PROJECT_NAME)

PROJECT_NAME:  delta-lake-diy-lab


In [5]:
project_number_output = !gcloud projects describe $PROJECT_ID | grep projectNumber | cut -d':' -f2 | xargs
PROJECT_NUMBER = project_number_output[0]
print("PROJECT_NUMBER: ", PROJECT_NUMBER)

PROJECT_NUMBER:  11002190840


In [6]:
DATA_LAKE_ROOT_PATH= f"gs://dll-data-bucket-{PROJECT_NUMBER}"
DELTA_LAKE_DIR_ROOT = f"{DATA_LAKE_ROOT_PATH}/delta-consumable"
print(DELTA_LAKE_DIR_ROOT)

gs://dll-data-bucket-11002190840/delta-consumable


### 4. File listing

In [7]:
!gsutil ls -r $DELTA_LAKE_DIR_ROOT

gs://dll-data-bucket-11002190840/delta-consumable/:
gs://dll-data-bucket-11002190840/delta-consumable/part-00000-0a06babc-6211-4fe4-ad01-3e1f9726830c-c000.snappy.parquet
gs://dll-data-bucket-11002190840/delta-consumable/part-00000-18ffd7b0-964d-4f1e-a648-0892a1d4373d-c000.snappy.parquet
gs://dll-data-bucket-11002190840/delta-consumable/part-00000-55d5ffb4-6b4a-4ead-8591-983dc631339f-c000.snappy.parquet
gs://dll-data-bucket-11002190840/delta-consumable/part-00000-5afc31da-4141-4f21-93e5-0ee8635aee9f-c000.snappy.parquet
gs://dll-data-bucket-11002190840/delta-consumable/part-00000-5f4a2030-2b43-4229-815a-55fca555d323-c000.snappy.parquet
gs://dll-data-bucket-11002190840/delta-consumable/part-00000-6486345e-6c92-4216-b9ed-ffb413e41b4d-c000.snappy.parquet
gs://dll-data-bucket-11002190840/delta-consumable/part-00000-687d9651-589c-4d6c-a8fd-df5d01770e85-c000.snappy.parquet
gs://dll-data-bucket-11002190840/delta-consumable/part-00000-6dbd9c8a-28bb-48a7-b1a9-5304b8f16bc2-c000.snappy.parquet
gs:/

In [8]:
!gsutil ls -r $DELTA_LAKE_DIR_ROOT/part* | wc -l

38


### 5. Optimize 
Delta Lake can improve the speed of read queries from a table by coalescing small files into larger ones through the "OPTIMIZE TABLE" command.
It is important to schedule periodic runs of this command for performance in a busy environment with too many small files.

Note: Optimize does not remove the small files, they still need to be deleted. It merely creates larger files that are indexed.

- Bin-packing optimization is idempotent, meaning that if it is run twice on the same dataset, the second run has no effect.

- Bin-packing aims to produce evenly-balanced data files with respect to their size on disk, but not necessarily number of tuples per file. However, the two measures are most often correlated.

- Python and Scala APIs for executing OPTIMIZE operation are available from Delta Lake 2.0 and above.

- Set Spark session configuration spark.databricks.delta.optimize.repartition.enabled=true to use repartition(1) instead of coalesce(1) for better performance when compacting many small files.

In [9]:
spark.sql("OPTIMIZE loan_db.loans_by_state_delta").show(truncate=False)

ivysettings.xml file not found in HIVE_HOME or HIVE_CONF_DIR,/etc/spark/conf/ivysettings.xml will be used
23/12/03 00:05:29 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+-------------------------------------------------+---------------------------------------------------------------------------------------------------------------------------------+
|path                                             |metrics                                                                                                                          |
+-------------------------------------------------+---------------------------------------------------------------------------------------------------------------------------------+
|gs://dll-data-bucket-11002190840/delta-consumable|{0, 0, {null, null, 0.0, 0, 0}, {null, null, 0.0, 0, 0}, 0, null, 0, 1, 1, false, 0, 0, 1701561939350, 0, 8, 0, null, null, 3, 3}|
+-------------------------------------------------+---------------------------------------------------------------------------------------------------------------------------------+



In [10]:
!gsutil ls -r $DELTA_LAKE_DIR_ROOT

gs://dll-data-bucket-11002190840/delta-consumable/:
gs://dll-data-bucket-11002190840/delta-consumable/part-00000-0a06babc-6211-4fe4-ad01-3e1f9726830c-c000.snappy.parquet
gs://dll-data-bucket-11002190840/delta-consumable/part-00000-18ffd7b0-964d-4f1e-a648-0892a1d4373d-c000.snappy.parquet
gs://dll-data-bucket-11002190840/delta-consumable/part-00000-55d5ffb4-6b4a-4ead-8591-983dc631339f-c000.snappy.parquet
gs://dll-data-bucket-11002190840/delta-consumable/part-00000-5afc31da-4141-4f21-93e5-0ee8635aee9f-c000.snappy.parquet
gs://dll-data-bucket-11002190840/delta-consumable/part-00000-5f4a2030-2b43-4229-815a-55fca555d323-c000.snappy.parquet
gs://dll-data-bucket-11002190840/delta-consumable/part-00000-6486345e-6c92-4216-b9ed-ffb413e41b4d-c000.snappy.parquet
gs://dll-data-bucket-11002190840/delta-consumable/part-00000-687d9651-589c-4d6c-a8fd-df5d01770e85-c000.snappy.parquet
gs://dll-data-bucket-11002190840/delta-consumable/part-00000-6dbd9c8a-28bb-48a7-b1a9-5304b8f16bc2-c000.snappy.parquet
gs:/

In [11]:
!gsutil ls -r $DELTA_LAKE_DIR_ROOT/part* | wc -l

38


Note that the parquet file count has not reduced. 

### 6. Restore to a prior version of the table

In [12]:
spark.sql("RESTORE TABLE loan_db.loans_by_state_delta TO VERSION AS OF 5").show(truncate=False)

+------------------------+--------------------------+-----------------+------------------+------------------+-------------------+
|table_size_after_restore|num_of_files_after_restore|num_removed_files|num_restored_files|removed_files_size|restored_files_size|
+------------------------+--------------------------+-----------------+------------------+------------------+-------------------+
|993                     |1                         |1                |1                 |7688              |993                |
+------------------------+--------------------------+-----------------+------------------+------------------+-------------------+



In [13]:
!gsutil ls -r $DELTA_LAKE_DIR_ROOT/part* | wc -l

38


In [14]:
!gsutil ls -r $DELTA_LAKE_DIR_ROOT/part* 

gs://dll-data-bucket-11002190840/delta-consumable/part-00000-0a06babc-6211-4fe4-ad01-3e1f9726830c-c000.snappy.parquet
gs://dll-data-bucket-11002190840/delta-consumable/part-00000-18ffd7b0-964d-4f1e-a648-0892a1d4373d-c000.snappy.parquet
gs://dll-data-bucket-11002190840/delta-consumable/part-00000-55d5ffb4-6b4a-4ead-8591-983dc631339f-c000.snappy.parquet
gs://dll-data-bucket-11002190840/delta-consumable/part-00000-5afc31da-4141-4f21-93e5-0ee8635aee9f-c000.snappy.parquet
gs://dll-data-bucket-11002190840/delta-consumable/part-00000-5f4a2030-2b43-4229-815a-55fca555d323-c000.snappy.parquet
gs://dll-data-bucket-11002190840/delta-consumable/part-00000-6486345e-6c92-4216-b9ed-ffb413e41b4d-c000.snappy.parquet
gs://dll-data-bucket-11002190840/delta-consumable/part-00000-687d9651-589c-4d6c-a8fd-df5d01770e85-c000.snappy.parquet
gs://dll-data-bucket-11002190840/delta-consumable/part-00000-6dbd9c8a-28bb-48a7-b1a9-5304b8f16bc2-c000.snappy.parquet
gs://dll-data-bucket-11002190840/delta-consumable/part-0

### 7. Vacuum - for deleting small files
Vacuum purges the small files that are not indexed.

In [ ]:
spark.sql("VACUUM loan_db.loans_by_state_delta").show(truncate=False)

In [ ]:
!gsutil ls -r $DELTA_LAKE_DIR_ROOT/part* | wc -l

In [ ]:
spark.sql("VACUUM loan_db.loans_by_state_delta RETAIN 0 HOURS")

Delta does not allow you to VACUUM with 0 retention unless you have the below property set-

In [ ]:
spark.conf.set("spark.databricks.delta.retentionDurationCheck.enabled" , False)

Now VACUUM should work..

In [ ]:
spark.sql("VACUUM loan_db.loans_by_state_delta RETAIN 0 HOURS")

In [ ]:
!gsutil ls -r $DELTA_LAKE_DIR_ROOT/part* | wc -l

In [ ]:
!gsutil ls -r $DELTA_LAKE_DIR_ROOT

### THIS CONCLUDES THIS LAB. DONT FORGET TO SHUT DOWN THE LAB RESOURCES.